<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/Assignment_7/ExtraCredit_Assignment_7e_XGBoost_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XGBoost_SparkML

## **Implementation Details and Discussion**

 
* In this colab I have implemented SparkML with XGBoost+ gridsearch and hyperparams 
* For that I have used Titanic Kaggle dataset. 
* I have performed preprocessing one data, then feature engineering before moving to Xgboost




# Spark Env Setup and Import Packages

In [ ]:
!pip install findspark

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'

import findspark
findspark.init()

import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Load Titanic Dataset

In [ ]:
spark = SparkSession\
        .builder\
        .appName("PySpark XGBOOST Titanic")\
        .getOrCreate()

In [ ]:
spark.sparkContext.addPyFile("YOUR_PATH/sparkxgb.zip")

In [ ]:
from sparkxgb import XGBoostEstimator

In [ ]:
schema = StructType(
  [StructField("PassengerId", DoubleType()),
    StructField("Survival", DoubleType()),
    StructField("Pclass", DoubleType()),
    StructField("Name", StringType()),
    StructField("Sex", StringType()),
    StructField("Age", DoubleType()),
    StructField("SibSp", DoubleType()),
    StructField("Parch", DoubleType()),
    StructField("Ticket", StringType()),
    StructField("Fare", DoubleType()),
    StructField("Cabin", StringType()),
    StructField("Embarked", StringType())
  ])

In [ ]:
df_raw = spark\
  .read\
  .option("header", "true")\
  .schema(schema)\
  .csv("YOUR_PATH/train.csv")

In [ ]:
df_raw.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survival|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|        1.0|     0.0|   3.0|Braund, Mr. Owen ...|  male|22.0|  1.0|  0.0|A/5 21171|   7.25| null|       S|
|        2.0|     1.0|   1.0|Cumings, Mrs. Joh...|female|38.0|  1.0|  0.0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [ ]:
df = df_raw.na.fill(0)

In [ ]:
sexIndexer = StringIndexer()\
  .setInputCol("Sex")\
  .setOutputCol("SexIndex")\
  .setHandleInvalid("keep")
    
cabinIndexer = StringIndexer()\
  .setInputCol("Cabin")\
  .setOutputCol("CabinIndex")\
  .setHandleInvalid("keep")
    
embarkedIndexer = StringIndexer()\
  .setInputCol("Embarked")\
  .setOutputCol("EmbarkedIndex")\
  .setHandleInvalid("keep")

# Vector Assembler

In [ ]:
vectorAssembler = VectorAssembler()\
  .setInputCols(["Pclass", "SexIndex", "Age", "SibSp", "Parch", "Fare", "CabinIndex", "EmbarkedIndex"])\
  .setOutputCol("features")

# Xgboost

In [ ]:
xgboost = XGBoostEstimator(
    featuresCol="features", 
    labelCol="Survival", 
    predictionCol="prediction"
)

In [ ]:
pipeline = Pipeline().setStages([sexIndexer, cabinIndexer, embarkedIndexer, vectorAssembler, xgboost])

In [ ]:
trainDF, testDF = df.randomSplit([0.8, 0.2], seed=24)

# Model and Training

In [ ]:
model = pipeline.fit(trainDF)

In [ ]:
model.transform(testDF).select(col("PassengerId"), col("prediction")).show()

+-----------+----------+
|PassengerId|prediction|
+-----------+----------+
|        1.0|       0.0|
|        4.0|       1.0|
|       14.0|       0.0|
|       15.0|       1.0|
|       20.0|       1.0|
|       28.0|       1.0|
|       34.0|       0.0|
|       38.0|       0.0|
|       50.0|       1.0|
|       52.0|       0.0|
|       59.0|       1.0|
|       60.0|       0.0|
|       82.0|       0.0|
|       94.0|       0.0|
|       96.0|       0.0|
|       99.0|       1.0|
|      104.0|       0.0|
|      105.0|       0.0|
|      107.0|       1.0|
|      116.0|       0.0|
+-----------+----------+
only showing top 20 rows

